In [9]:
import requests #pip install requests
from xml.etree import ElementTree #pip install xml.etree
import datetime as dt #pip install datetime

import io
import pdfminer #pip install pdfminer
from pdfminer.high_level import extract_text  #pip install pdfminer.six





In [23]:
from flask import Flask, Response


app = Flask(__name__)

@app.route('/')
def index():
    return 'AI ARTCILES'


@app.route('/get_data')
def get_data():
    articles = []
    arxiv_url = "http://export.arxiv.org/api/query?"
    research_type = {"search_query": "all:ai", "max_results" : 5}

    rep = requests.get(arxiv_url, params=research_type)
    root = ElementTree.fromstring(rep.content)
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        published = entry.find('{http://www.w3.org/2005/Atom}published').text[0:10] 

        articles.append([entry.find('{http://www.w3.org/2005/Atom}id').text, published])

    for article in articles: # pour trier les artciles dans l'ordre chronologik
        article[1] = dt.datetime.strptime(article[1], '%Y-%m-%d')
    articles.sort(key=lambda x: x[1], reverse = True)
    articles = [article[0] for article in articles] #On les a rangé par odre chrono mais pour cette route on n'a pas besoin des dates de publi
    
    html = "<br>".join(articles)
    return Response(html, mimetype='text/html')


    


@app.route('/articles')
def articles():
    articles = []
    arxiv_url = "http://export.arxiv.org/api/query?"
    research_type = {"search_query": "all:ai", "max_results" : 5}

    rep = requests.get(arxiv_url, params=research_type)
    root = ElementTree.fromstring(rep.content)
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        title = entry.find('{http://www.w3.org/2005/Atom}title').text
        abs = entry.find('{http://www.w3.org/2005/Atom}summary').text
        auteur = entry.find('{http://www.w3.org/2005/Atom}author').find('{http://www.w3.org/2005/Atom}name').text
        published = entry.find('{http://www.w3.org/2005/Atom}published').text[0:10] 
        link = entry.find('{http://www.w3.org/2005/Atom}id').text
        id = link[21:]
        urlpdf = f"http://arxiv.org/pdf/{id}"

        articles.append([id, link, auteur, title, published, urlpdf, abs])
    
    
    for article in articles: # pour trier les artciles dans l'ordre chronologik
        article[4] = dt.datetime.strptime(article[4], '%Y-%m-%d')
    articles.sort(key=lambda x: x[4], reverse = True)

    html = "<br>".join([f"ID: {a[0]}<br>Link: {a[1]}<br>Authors: {a[2]}<br>Title: {a[3]}<br>Published: {a[4]}<br>PDF: {a[5]}<br>Summary: {a[6]}<br><br>" for a in articles])
    return Response(html, mimetype='text/html')


@app.route('/article/<id>')
def article_content(id):
    urlpdf = f"http://arxiv.org/pdf/{id}"
    response = requests.get(urlpdf)

    text = extract_text(io.BytesIO(response.content))

    # Convertir les sauts de ligne en balises <br> pour l'affichage HTML
    html_content = "Lien pour accéder au pdf direct : " + urlpdf +'<br>'+'<br>'+'<br>' + text.replace('\n', '<br>')

    return Response(html_content, mimetype='text/html')




@app.route('/ml')
def ml_method():
    return 

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2024 13:52:23] "GET /article/2402.07632v1 HTTP/1.1" 200 -
